# BINF GU 4002: Machine Learning for Healthcare, Spring 2025
# Assignment \#1: Classification with Supervised Learning
## DUE: 11:59 PM, Tuesday, February 18, 2025

This assignment is a primer on binary classifiction with supervised learning. You will be implementing two classifiers and using them to train and evaluate models on a real world tabular dataset.

**Instructions for accessing the dataset are below**--please make sure to read them. To save you some time, we have handled preprocessing the data for you.

**<font color="red">Please make sure that your Notebook runs (no running cells out of order!) and that your written answers are formatted using </font>$\LaTeX$<font color="red"> in `markdown` cells. When submitting, please name your files `{UNI}_binf4008_mlh_assignment_1.{filetype}` and submit both a `.ipynb` and `.html` version of your Jupyter notebook.</font>**

#### The Machine Learning Task

You will be implementing two workhorse classification models--[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) and [Naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)--using base `numpy` in an [object oriented programmatic](https://en.wikipedia.org/wiki/Object-oriented_programming) way.

In practice, training a classifier using [`LogisticRegression`](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html) from `sklearn.linear_model` can be done with a handful of lines of code (same goes for [Naïve Bayes](https://scikit-learn.org/1.5/modules/naive_bayes.html)):

```
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_train)
```

**The exercise here is to look under the proverbial "hood" of these two models.** We provide skeleton code and comments to help guide you.

#### The Clinical Problem and Data

We will be using the [Heart Disease](https://archive.ics.uci.edu/dataset/45/heart+disease) dataset to detect the presence of heart disease from tabular features. The dataset was used in "[International application of a new probability algorithm for the diagnosis of coronary artery disease](https://www.sciencedirect.com/science/article/pii/0002914989905249)" by Detrano et al. published in the *American Journal of Cardiology* in 1989 and sourced from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/). You can download the files directly from the UCI ML Repository or use the [`ucimlrepo`](https://github.com/uci-ml-repo/ucimlrepo) package.

From the UCI Repository:

> This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them.  In particular, the Cleveland database is the only one that has been used by ML researchers to date.  The "goal" field refers to the ***presence of heart disease in the patient.***  It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply attempting to ***distinguish presence (values 1,2,3,4) from absence (value 0).***

#### Evaluation

Once you train your models, you have to generate performance metrics on the held-out test set. However, rather than generating a point estimate for the performance, **we are going to acquire confidence intervals by bootstrapping the test set** (see [Dr. Raschka's primer on this here](https://sebastianraschka.com/blog/2016/model-evaluation-selection-part2.html)). Not only is getting some sort of uncertainty estimate of our performance good science and reproducibility practice, it is becoming a requirement when disseminating your machine learning research. From the [AAAI Reproducibility Checklist](https://aaai.org/conference/aaai/aaai-23/reproducibility-checklist/):

> Analysis of experiments goes beyond single-dimensional summaries of performance (e.g., average; median) to include measures of variation, confidence, or other distributional information. (yes/no)

#### Best of Luck!

In [1]:
import os

import numpy as np
np.random.seed(0)
import pandas as pd
import matplotlib.pyplot as plt

# from ucimlrepo import fetch_ucirepo # uncomment if you are using the ucimlrepo package.
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

## [45 Points] Question 1: Building the Naïve Bayes Classifier

Our goal in Naïve Bayes is to get probabilities a feature vector $\mathbf{x}=(x_1,\ldots{},x_n)$ will belong to $K$ classes $C_k$ via conditional probabilities $P(C_k|\mathbf{x})$. Questions 1.1 - 1.3 will be written (create a `markdown` cell) and 1.4 will be a code implementation.

Starting with posterior $P(C_k|\mathbf{x})$, prior $P(C_k)$, likelihood $P(\mathbf{x}|C_k)$ and evidence $P(\mathbf{x})$, and using the [Chain Rule of Probability](https://en.wikipedia.org/wiki/Chain_rule_(probability)) and the [Law of Total Probability](https://en.wikipedia.org/wiki/Law_of_total_probability), do the following:

#### [5 Points] 1.1: Explain the Naïve Bayes assumption. Why is it "naïve?"

Hint: You should discuss [conditional independence](https://en.wikipedia.org/wiki/Conditional_independence).

Answer:

We assume that all features are conditionally independent of each other given the class label.

#### [10 Points] 1.2: Derive the expression below, starting with Bayes' Theorem:
$$P(C_k|\mathbf{x})=\frac{1}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}P(C_k)\prod^{n}_{i=1}P(x_i|C_k)$$

Hints:
- From Bayesian Probability: $\text{posterior}=\frac{\text{prior}\times{\text{likelihood}}}{\text{evidence}}$.
- Think about what the denominator on the expression on the right means in context of the Law of Total Probability
- $P(A,B)=P(B)P(A|B)$
- $P(C_k,\mathbf{x})=P(C_k,x_1, x_2,\ldots{}, x_n)$
- You need the Naïve Bayes Assumption.

Answer:

$$P(C_k|\mathbf{x})=\frac{P(C_k,\mathbf{x})}{P(\mathbf{x})}=\frac{P(C_k,x_1, x_2,\ldots{}, x_n)}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}=\frac{P(C_k)P(x_1,x_2,\ldots{},x_n|C_k)}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}=\frac{P(C_k)\prod^{n}_{i=1}P(x_i|C_k)}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}$$

#### [10 Points] 1.3: We will implement a [Gaussian Naïve Bayes algorithm](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) using [Maximum Likelihood Estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) for our classification task. Starting from the expression you derviced in Question 1.2, explain how you will build your classifier by deriving the decision rule $\text{argmax}_{y\in\{C_1,\ldots{},C_k\}}\log(C_k|\mathbf{x})$. 

Hints:
- We are trying to select the class $C_k$ that maximizes the posterior probability $P(C_k|\mathbf{x})$.
- What are we assuming about the likelihood of the features? We are assuming that each feature follows a Gaussian distribution *class-dependent* mean and variance.
    - What does this mean with respect to $P(x_i|C_k)$? Think about the Probability Density Function for the Gaussian distribution.
- Since $\frac{1}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}$ remains constant, you can ignore it during the maximization step.
- A useful property of logarithms is that $\log(\prod_{i}P(x_i))=\sum_{i}\log(P(x_i))$. This is part of thr eason why we maximize the log-likelihood $\log(P(C_k|\mathbf{x}))$.

Answer:

We want to find $k$ that maximize $P(C_k|\mathbf{x})=\frac{P(C_k)\prod^{n}_{i=1}P(x_i|C_k)}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)}$. Since the denominator remains constant and log transfrom is monotonously increasing, all we need to maximize is $(\log P(C_k)+\sum^n_{i=1}\log P(x_i|C_k))$.  $$k = \text{argmax} P(C_k|\mathbf{x}) = \text{argmax} \frac{P(C_k)\prod^{n}_{i=1}P(x_i|C_k)}{\sum_{k}P(C_k)P(\mathbf{x}|C_k)} = \text{argmax} \bigg(\log P(C_k)+\sum^n_{i=1}\log P(x_i|C_k)\bigg) $$ 

We assume that each feature follows a Gaussian distribution class-dependent mean and variance. 

$$x_i|C_k \sim N(\mu_{ik},\sigma_{ik}^2)$$ 

Suppose we have $m$ samples, each sample has $n$ features. The mean and variance can be estimated from data (maximum likelihood estimator of gaussian).

$$ \hat{\mu_{k}} = \frac{\sum^m_{j=1}x_j\cdot I(y_j = C_k)}{\sum^m_{j=1}I(y_j = C_k)}$$ 

$$ \hat{\sigma^2_{k}} = \frac{\sum^m_{j=1}(x_j-\hat{\mu_{k}})^2\cdot I(y_j = C_k)}{\sum^m_{j=1}I(y_j = C_k)}$$ 

From MLE of class-dependent mean and variance, we can calculate $\log P(x_i|C_k)$

$$ \log P(x_i|C_k) = \log (\frac{1}{\sqrt{2\pi\hat{\sigma^2_{ik}}}}\exp(\frac{(x_i-\hat{\mu_{ik}})}{2\hat{\sigma^2_{ik}}})) =  -\frac{1}{2}\log(2\pi\hat{\sigma_{ik}^2})+ \frac{(x_i-\hat{\mu_{ik}})^2}{2\hat{\sigma_{ik}^2}}$$


The class prior $P(C_k)$ can be estimated directly from the label frequency.

$$ \log P(C_k) = \log \frac{\sum^{m}_{i=1}I(y_i=C_k)}{m}$$


Combine two log-likelihood,

$$ k =  \text{argmax} \bigg(\log P(C_k)+\sum^n_{i=1}\log P(x_i|C_k)\bigg)$$

$$ \hat{y} = C_k $$

So we first estimate the mean and variance for each class, then calculate the log prior and log likelihood. By summing them, we obtain the log posterior likelihood for each class. The class with the maximun posterior likelihood is chosen as the predicted class.









#### [20 Points] 1.4: Implement a Naïve Bayes classifier with the skeleton code below.

#### You are allowed to reference the following materials:
- Lecture slides and notes from our course.
- The Wikipedia articles on [Naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier), the [Chain Rule of Probability](https://en.wikipedia.org/wiki/Chain_rule_(probability)) and the [Law of Total Probability](https://en.wikipedia.org/wiki/Law_of_total_probability).
- Dr. Kilian Q. Weinberger's [course notes](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote05.html) and lecture recording.
- Dr. Michael Collin's [notes on Naïve Bayes and Expectation Maximization](http://www.cs.columbia.edu/~mcollins/em.pdf).
- [Scikit-Learn documentation on Naïve Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html).

In [2]:
class NaiveBayes:
    
    def __init__(self):

        self.class_probs = None        # P(C_k), the prior
        self.class_means = None
        self.class_variances = None
        self.classes = None
    
    def fit(self, X, y):

        # Number of samples and features  
        n_samples, n_features = X.shape
        # Get unique class labels
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # Use np.zeros() to initialize the following:
        # class_probs with shape n_classes
        # class_means with shape (n_classes, n_features)
        # class_variances with (n_classes, n_features)
        self.class_probs = np.zeros(n_classes)
        self.class_means = np.zeros((n_classes, n_features))
        self.class_variances = np.zeros((n_classes, n_features))
        
        # X_c: index the features that belong to a class c (hint: you should use the value of y to index X)
        # class_probs: number of rows in X_c / # of samples in X
        # class_means is the mean of X_c along the 0th axis
        # class_var is the variance of X_c aong the 0th axis
        for i, c in enumerate(self.classes):

            X_c = np.array([X[j] for j in range(n_samples) if y[j] == c])
            self.class_probs[i] = X_c.shape[0] / n_samples
            self.class_means[i, :] = np.mean(X_c, axis=0)
            self.class_variances[i, :] = np.var(X_c, axis=0)
    
    def _calculate_log_likelihood(self, X):  

        # get the number of samples. remember, X.shape = (n_samples, n_features). 
        n_samples = X.shape[0]
        n_features = X.shape[1]
        # instantiate the likelihood with an array of zeros of shape (n_samples, n_classes). You can do this using np.zeros()
        likelihood = np.zeros((n_samples, len(self.classes)))
        
        # Iterate over the classes
        for i, _ in enumerate(self.classes):

            # get the mean and variance for the current class (by indexing class_means[i,:] and class_variances[i,:])
            mean = self.class_means[i, :]
            var = self.class_variances[i, :]
            
            # Calculate the Gaussian Likelihood and populate the [:,i]-th slice of likelihood array with the current class likelihood
            likelihood[:, i] = np.sum(-0.5 * np.log(2 * np.pi * var) - 0.5 * ((X - mean) ** 2) / var, axis=1)
        
        return likelihood
    
    def _calculate_log_posterior(self, X) :

        # log(posterior) = log(prior) + log(likelihood)
        prior = np.log(self.class_probs)
        likelihood = self._calculate_log_likelihood(X)
        posterior = prior + likelihood

        return posterior

    def predict(self, X):

        log_posterior = self._calculate_log_posterior(X)
        
        return self.classes[np.argmax(log_posterior, axis=1)]
    
    def predict_proba(self, X):

        log_posterior = self._calculate_log_posterior(X)
        
        posterior = np.exp(log_posterior)
        probabilities = posterior / np.sum(posterior, axis=1, keepdims = True)
        
        return probabilities

# [45 Points] Question 2: Building the Standard Logistic Regression Classifier

We are going to build toward a Logistic Regression classifier. This is one of the most important machine learning models that is used widely in healthcare and other industries.

Unlike the Naïve Bayes classifier, Logistic Regression has an optimization step. If you feel particularly lost in this question, start by reading Sections 5.1, 52, 5.4, and 5.6 in the [Logistic Regression chapter](https://web.stanford.edu/~jurafsky/slp3/5.pdf) in Dr. Jrafsky and Dr. Martin's *Speech and Language Processing*.

#### [10 Points] Question 2.1: Derive the log-loss minimization $L(\hat{y},y)$ for Logistic Regression. Start with the idea that we can model binary labels $P(y=1 \vert \mathbf{x}) \sim \text{Bernoulli}(\sigma(\mathbf{w}^T \mathbf{x}))$, where $\sigma{(\mathbf{x})}=\frac{1}{1+e^{-\mathbf{x}}}$ is the sigmoid function.

Hint: take the following steps.
- Start with $\hat{y}={\sigma(\mathbf{w}^T\mathbf{x})}$
- Bernoulli means that $P(y=1\vert{{\mathbf{x}}})=\hat{y}^{y}(1-\hat{y})^{1-y}$.
- $L(\hat{y},y)=-\log{\left(P(y=1\vert{\mathbf{x}})\right)}$. Replace $\hat{y}$ in terms of $\mathbf{x}$ and expand the expression.

Answer:

$$L(\hat{y},y)=-\log{\left(P(y=1\vert{\mathbf{x}})\right)} = -\log (\hat{y}^{y}(1-\hat{y})^{1-y}) = -\big[y\log\hat{y}+(1-y)\log(1-\hat{y})\big] = -\big[y\log\sigma(\mathbf{w}^T\mathbf{x})+(1-y)\log(1-\sigma(\mathbf{w}^T\mathbf{x}))\big]$$



#### [10 Points] Question 2.2: Get the derivative of $\sigma{(\mathbf{w}^T \mathbf{x})}$ with respect to $w_j\in{}\mathbf{w}$.

Hint: Plug in $\mathbf{w}^T \mathbf{x}$ to $\sigma{(\mathbf{x})}=\frac{1}{1+e^{-\mathbf{x}}}$. Then, either use the quotient rule of partial derivatives:

$$\frac{\partial}{\partial{x}}\frac{f(x,y)}{g(x,y)}=\frac{\frac{\partial{f(x,y)}}{\partial{x}}g(x,y)-f(x,y)\frac{\partial{g(x,y)}}{\partial{x}}}{\left(g(x,y)\right)^2}$$

or express $\sigma{(\mathbf{x})}$ as $(1+e^{-\mathbf{x}})^{-1}$ and use the chain rule.

Answer:

$$\frac{\partial\sigma{(\mathbf{w}^T \mathbf{x})}}{\partial{\mathbf{w}}}=\frac{\partial\sigma{(\mathbf{w}^T \mathbf{x})}}{\partial(\mathbf{w}^T\mathbf{x})}\frac{\partial(\mathbf{w}^T\mathbf{x})}{\partial{\mathbf{w}}}=-\frac{1}{(1+e^{-\mathbf{w}^T \mathbf{x}})^2}\cdot e^{-\mathbf{w}^T \mathbf{x}} \cdot \mathbf{x}$$

#### [10 Points] Question 2.3: Derive a gradient descent update to optimize for $\mathbf{w}$.

Hint: This is the derivative of the expression you derived from Question 2.1 ($L(\hat{y},y)$) with respect to $w_j\in{\mathbf{w}}$.

$$\frac{\partial L(\hat{y},y)}{\partial \mathbf{w}} = -\big[y\cdot (1+e^{-\mathbf{w}^T\mathbf{x}})\cdot -\frac{1}{(1+e^{-\mathbf{w}^T\mathbf{x}})^2}\cdot (e^{-\mathbf{w}^T\mathbf{x}}) \cdot \mathbf{x} + (1-y)\cdot \frac{(1+e^{-\mathbf{w}^T\mathbf{x}})}{e^{-\mathbf{w}^T\mathbf{x}}}\cdot \frac{1}{(1+e^{-\mathbf{w}^T\mathbf{x}})^2}\cdot e^{-\mathbf{w}^T\mathbf{x}} \cdot \mathbf{x} \big] = (\frac{1}{1+e^{-\mathbf{w}^T\mathbf{x}}}-y)\mathbf{x}=(\hat{y}-y)\mathbf{x}$$

#### [15 Points] Question 2.4: Build a Logistic Regression classifier with the skeleton code below.

#### You are allowed to reference the following materials:
- Lecture slides and notes from our course.
- The [Logistic Regression chapter](https://web.stanford.edu/~jurafsky/slp3/5.pdf) in Dr. Jrafsky and Dr. Martin's *Speech and Language Processing*.
- The Wikipedia articles on [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression).
- Dr. Kilian Q. Weinberger's [course notes](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote06.html) and lecture recording.
- [Scikit-Learn documentation on Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).


In [3]:
class LogisticRegression :

    # you will populate self.classes, self.w with the dimensions of the training data in the self.fit() functionj
    # losses will be populated during training with the self.fit() function.
    def __init__(self) :

        self.classes = None
        self.w = None
        self.losses = []

    def sigmoid(self, x) :

        return 1 / (1 + np.exp(-x))

    # This should be your answer from question 2.1
    # Adding a small eps value in logs (e.g. np.log(y + eps)) can help with numeric stability.
    # you can divide the loss by the number of samples i.e. len(y_true)
    def loss(self, y_true, y_pred, eps = 10e-6):

        assert y_true.shape == y_pred.shape

        return -y_true * np.log(y_pred + eps) - (1 - y_true) * np.log(1 - y_pred + eps)
    
    def predict_proba(self, x) :
        
        return self.sigmoid(np.matmul(x, self.w))

    def predict(self, x) :
        y_pred_probs = self.predict_proba(x)
        # one hot encode y_preds
        y_preds = np.expand_dims(np.argmax(y_pred_probs, axis = 1), axis = -1)
        return np.column_stack([1 - y_preds, y_preds])
    
    def fit(self, x, y_true, eta = 0.1, C = 0.01, n_iterations = 10000, eps = 10e-6) :

        # if you labels is of shape (n_samples,) rather than (n_samples,2), this loop will handle it
        if len(y_true.shape) == 1 :
            y_true = np.expand_dims(y_true, axis = -1)
            y_true = np.column_stack([1-y_true, y_true])

        # Number of samples and features
        n_features = x.shape[1]
        # Get unique class labels
        self.classes = np.unique(y_true)
        n_classes = len(self.classes)
        self.w = np.zeros((n_features, n_classes))

        self.losses = []
        for _ in range(n_iterations) :

            y_pred = self.predict_proba(x)
            # the gradient should be your answer from question 2.3
            gradient = np.dot(x.T, y_pred - y_true)/len(y_true) + C * self.w
            self.losses.append(np.mean(self.loss(y_true, y_pred)))
            self.w -= eta * gradient

# [10 Points] Question 3: Training and Evaluating the Classifiers

Now that you have done the hard work of building your models, it's time to train!

We have already supplied you with starter code to get your `X_train`, `y_test`, etc (don't worry, you'll get plenty of opportunity for data cleaning and processing in later assignments). Instantiate model objects for both `NaiveBayes` and `LogisticRegression` and use their respective `.fit()`, `.predict_proba()`, and `.predict()` methods to train your models and generate predictions on the test set.

As we discussed in the preamble, it's important to generate distributional estimates of your performance (rather than point estimates). There are several ways to get these confidence intervals; however, we are going to consider a fairly lightweight method of doing this by resampling the test set to generate our metrics--if you find this task daunting, we suggest starting with the [definition of bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) (e.g. how do we sample with replacement with `numpy`), what a 95% [confidence interval](https://en.wikipedia.org/wiki/Confidence_interval) is supposed to represent, and [the primer by Dr. Sebastian Rashka](https://sebastianraschka.com/blog/2022/confidence-intervals-for-ml.html) (specifically [Method 3](https://sebastianraschka.com/blog/2022/confidence-intervals-for-ml.html#method-3-bootstrapping-the-test-set-predictions)).

#### [10 Points] 3.1: Using the starter code below, train both a `NaiveBayes` and `LogisticRegression` model and generate 95% confidence intervals for Accuracy, AUPRC, and AUROC by bootstrapping the test set. You may use `sklearn.metrics` to generate your scores.

In [4]:
# STARTER CODE

# If you downloaded the data and label files locally, replace the file paths below.
DATA_PATH, LABEL_PATH = os.path.join('data', 'heart_disease_data.csv'), os.path.join('data', 'heart_disease_labels.csv')

if os.path.isfile(DATA_PATH) and os.path.isfile(LABEL_PATH) :

    X = pd.read_csv(DATA_PATH)
    y = pd.read_csv(LABEL_PATH)

# If you are using the UCI ML Repo package instead:
else :

    from ucimlrepo import fetch_ucirepo

    heart_disease = fetch_ucirepo(id = 45)
    X = heart_disease.data.features
    y = heart_disease.data.targets

# Feature descriptions from the UCI Machine Learning Repository documentation:
# cp (categorical): chest pain type (1: typical angina, 2: atypical angina, 3: non-anginal pain, 4: asymptomatic)
# trestbps (integer): resting blood pressure (in mm Hg on admission to the hospital)
# chol (integer): serum cholestoral in mg/dl
# fbs (categorical): (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
# restecg (categorical): resting electrocardiographic results (0: normal, 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), 2: showing probable or definite left ventricular hypertrophy by Estes' criteria)
# thalach (integer): maximum heart rate achieved
# exang (categorical): exercise induced angina (1 = yes; 0 = no)
# oldpeak (integer): ST depression induced by exercise relative to rest
# slope (categorical): the slope of the peak exercise ST segment (1: upsloping, 2: flat, 3: downsloping)
# ca (integer): number of major vessels (0-3) colored by flourosopy
# thal (categorical): 3 = normal; 6 = fixed defect; 7 = reversable defect

# Convert labels to binary values.
y = y['num'].to_numpy()
y[y!=0] = 1

data_type_by_columns = {"age":"integer",
                        "sex":"categorical",
                        "cp": "categorical",
                        "trestbps": "integer",
                        "chol": "integer",
                        "fbs": "categorical",
                        "restecg": "categorical",
                        "thalach": "integer",
                        "exang": "categorical",
                        "oldpeak": "integer",
                        "slope": "categorical",
                        "ca": "integer",
                        "thal": "categorical"}

columns_to_convert_to_dummies = [column_name for column_name,data_type in data_type_by_columns.items() if (data_type == 'categorical') and len(set(X[column_name])) > 2]

for column_name in columns_to_convert_to_dummies :
    current_dummies = pd.get_dummies(X[column_name], prefix = column_name)
    X = pd.concat([X, current_dummies], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print(f'Label counts: train {Counter(y_train)}, test {Counter(y_test)}')

median_imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
median_imputer.fit(X_train)
X_train, X_test = median_imputer.transform(X_train), median_imputer.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

Label counts: train Counter({0: 129, 1: 113}), test Counter({0: 35, 1: 26})


In [5]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,...,cp_4,restecg_0,restecg_1,restecg_2,slope_1,slope_2,slope_3,thal_3.0,thal_6.0,thal_7.0
0,63,1,1,145,233,1,2,150,0,2.3,...,False,False,False,True,False,False,True,False,True,False
1,67,1,4,160,286,0,2,108,1,1.5,...,True,False,False,True,False,True,False,True,False,False
2,67,1,4,120,229,0,2,129,1,2.6,...,True,False,False,True,False,True,False,False,False,True
3,37,1,3,130,250,0,0,187,0,3.5,...,False,True,False,False,False,False,True,True,False,False
4,41,0,2,130,204,0,2,172,0,1.4,...,False,False,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,...,False,True,False,False,False,True,False,False,False,True
299,68,1,4,144,193,1,0,141,0,3.4,...,True,True,False,False,False,True,False,False,False,True
300,57,1,4,130,131,0,0,115,1,1.2,...,True,True,False,False,False,True,False,False,False,True
301,57,0,2,130,236,0,2,174,0,0.0,...,False,False,False,True,False,True,False,True,False,False


In [6]:
NaiveBayes_model = NaiveBayes()
NaiveBayes_model.fit(X_train, y_train)
y_pred = NaiveBayes_model.predict(X_test)
y_pred_proba = NaiveBayes_model.predict_proba(X_test)

# generate 95% confidence interval for the accuracy, AUPRC, AUROC of the model
n_bootstraps = 1000
bootstrap_accuracies = []
bootstrap_auprc = []
bootstrap_auroc = []
for _ in range(n_bootstraps):
    indices = np.random.choice(range(len(y_test)), len(y_test), replace=True)
    bootstrap_accuracies.append(np.mean(y_test[indices] == y_pred[indices]))
    bootstrap_auprc.append(average_precision_score(y_test[indices], y_pred_proba[indices, 1]))
    bootstrap_auroc.append(roc_auc_score(y_test[indices], y_pred_proba[indices, 1]))
print('NaiveBayes model accuracy: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_accuracies), np.percentile(bootstrap_accuracies, 2.5), np.percentile(bootstrap_accuracies, 97.5)))
print('NaiveBayes model AUPRC: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_auprc), np.percentile(bootstrap_auprc, 2.5), np.percentile(bootstrap_auprc, 97.5)))
print('NaiveBayes model AUROC: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_auroc), np.percentile(bootstrap_auroc, 2.5), np.percentile(bootstrap_auroc, 97.5)))


NaiveBayes model accuracy: 0.801, 95% CI: [0.689,0.902]
NaiveBayes model AUPRC: 0.841, 95% CI: [0.712,0.937]
NaiveBayes model AUROC: 0.837, 95% CI: [0.726,0.936]


In [7]:
# logistic regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
y_pred_proba = logistic_model.predict_proba(X_test)

# generate 95% confidence interval for the accuracy, of the model
n_bootstraps = 1000
bootstrap_accuracies = []
bootstrap_auprc = []
bootstrap_auroc = []
for _ in range(n_bootstraps):
    indices = np.random.choice(range(len(y_test)), len(y_test), replace=True)
    bootstrap_accuracies.append(np.mean(y_test[indices] == y_pred[indices,1]))
    bootstrap_auprc.append(average_precision_score(y_test[indices], y_pred_proba[indices][:,1]))
    bootstrap_auroc.append(roc_auc_score(y_test[indices], y_pred_proba[indices][:,1]))

print('Logistic Regression Accuracy: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_accuracies),np.percentile(bootstrap_accuracies, 2.5), np.percentile(bootstrap_accuracies, 97.5)))
print('Logistic Regression AUPRC: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_auprc), np.percentile(bootstrap_auprc, 2.5), np.percentile(bootstrap_auprc, 97.5)))
print('Logistic Regression AUROC: {:.3f}, 95% CI: [{:.3f},{:.3f}]'.format(np.mean(bootstrap_auroc), np.percentile(bootstrap_auroc, 2.5), np.percentile(bootstrap_auroc, 97.5)))

Logistic Regression Accuracy: 0.786, 95% CI: [0.688,0.885]
Logistic Regression AUPRC: 0.839, 95% CI: [0.704,0.940]
Logistic Regression AUROC: 0.864, 95% CI: [0.767,0.943]
